In [3]:
import pandas as pd
import mysql.connector

In [7]:
conn_mysql = mysql.connector.connect(host='10.100.10.29',
                                            user="himanshu", 
                                            passwd="yNVb@123_#@",
                                            database = 'ekodb_fingpay')

InterfaceError: 2003: Can't connect to MySQL server on '10.100.10.29:3306' (10060 A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond)

In [3]:
query = 'desc bank_detail'
df = pd.read_sql(query, conn_mysql)
df

,Field,Type,Null,Key,Default,Extra
0,BANK_DETAIL_ID,int(11),NO,PRI,None,auto_increment
1,BANK_NAME,varchar(100),YES,,None,
2,NARRATION_PREFIX,varchar(10),YES,MUL,None,
3,AccNumLength,varchar(50),YES,,None,
4,ValidateAccountLength,int(11),YES,,0,
5,IMPS_STATUS,int(11),YES,,0,
6,STATUS,int(11),YES,,1,
7,NEFT_Status,int(11),YES,,0,
8,STATIC_IFSC,char(11),YES,,None,
9,IFSC_STATUS,char(11),YES,,None,


In [8]:
df.to_csv('bank_detail.csv', index=False)

In [4]:
# aeps fino fingpay failed tx
query_1 = f"select distinct(wt.holdtransactionid) as ekotrxnid, wts.description as status,ap.destination, date(wt.txtime) as date, bd.bank_name, \
wt.narration as message from wallettransaction wt \
left join wallettransactionstatus wts on wt.status = wts.statusid \
left join aepstrxndetail ap on (ap.ekotrxnid=wt.holdtransactionid) \
left join bank_detail bd on ap.bankid = bd.bank_detail_id \
where wt.txtime >= '2021-10-01 00:00:00' and wt.txtime < '2022-01-01 00:00:00' and wt.typeid in (345) and wt.status = 7 \
order by ekotrxnid"
df2 = pd.read_sql(query_1, conn_mysql)

In [3]:
# customer address
query_1 = '''select csp.cspcode, cad.city, cad.state, cad.zip from ekocsp csp left join customer_address_detail cad on 
csp.customerid = cad.customer_id where csp.cspcode is not null
'''
df = pd.read_sql(query_1, conn_mysql)
df

,cspcode,city,state,zip
0,99020000,NEW DELHI,Delhi,110019
1,99020006,GURGAON,Haryana,122001
2,99020005,GURGAON,Haryana,122001
3,99020002,GURGAON,Haryana,122001
4,99010389,PATNA,Bihar,800001
...,...,...,...,...
672317,10162029,DELHI,DELHI,110059
672318,10162030,DELHI,DELHI,110059
672319,10162031,DELHI,DELHI,110059
672320,10162032,Chandauli,Uttar Pradesh,232101


In [5]:
df.to_csv('all_address.csv', index=False)

In [23]:
# aeps fino fingpay total tx
query_1 = '''select distinct(wt.holdtransactionid) as trxnid, sum(wt.amount) as amount, wts.description as status, 
ap.destination, wt.txtime, bd.bank_name, csp.cspcode,
wt.narration as message from wallettransaction wt 
left join wallettransactionstatus wts on wt.status = wts.statusid 
left join aepstrxndetail ap on (ap.ekotrxnid=wt.holdtransactionid) 
left join bank_detail bd on ap.bankid = bd.bank_detail_id 
left join ekocsp csp on wt.originationid = csp.customerid
where wt.txtime >= '2021-11-01' and wt.txtime < '2021-12-01' and wt.typeid in (345)
group by destination, status, bd.bank_name, trxnid, wt.txtime
'''
df2 = pd.read_sql(query_1, conn_mysql)
df2

,trxnid,amount,status,destination,txtime,bank_name,cspcode,message
0,2818320107,510.0,Fail,26,2021-11-01 10:04:26,Airtel Payments Bank,99077704,If client level validations are done then Aadh...
1,2818704235,1000.0,Fail,26,2021-11-01 16:08:12,Airtel Payments Bank,20236367,Insufficient fund
2,2818708917,450.0,Fail,26,2021-11-01 16:13:33,Airtel Payments Bank,20236367,"Fingerprint did not matched with Aadhaar, plea..."
3,2818752536,10000.0,Fail,26,2021-11-01 17:01:31,Airtel Payments Bank,99028476,Please do eKYC before initiating transaction
4,2819199185,500.0,Fail,26,2021-11-02 08:07:37,Airtel Payments Bank,20390048,Insufficient fund
...,...,...,...,...,...,...,...,...
414201,2838914225,1010.0,Success,29,2021-11-28 15:23:13,Yes Bank,10051001,AEPS Transaction Success
414202,2839392313,3000.0,Success,29,2021-11-29 10:25:10,Yes Bank,99010442,AEPS Transaction Success
414203,2839554684,5000.0,Success,29,2021-11-29 13:07:08,Yes Bank,99056053,AEPS Transaction Success
414204,2839899838,1000.0,Success,29,2021-11-29 19:36:30,Yes Bank,21810000,AEPS Transaction Success


In [24]:
df2 = df2.replace({'destination':{26:'Fingpay', 29:'FINO'}})

In [25]:
df2 = df2.merge(df, on='cspcode', how='left').drop_duplicates(subset='trxnid')

In [26]:
df2

,trxnid,amount,status,destination,txtime,bank_name,cspcode,message,city,state,zip
0,2818320107,510.0,Fail,Fingpay,2021-11-01 10:04:26,Airtel Payments Bank,99077704,If client level validations are done then Aadh...,Ghaziabad,Uttar Pradesh,201010
1,2818704235,1000.0,Fail,Fingpay,2021-11-01 16:08:12,Airtel Payments Bank,20236367,Insufficient fund,Jammu,Jammu and Kashmir,180004
2,2818708917,450.0,Fail,Fingpay,2021-11-01 16:13:33,Airtel Payments Bank,20236367,"Fingerprint did not matched with Aadhaar, plea...",Jammu,Jammu and Kashmir,180004
3,2818752536,10000.0,Fail,Fingpay,2021-11-01 17:01:31,Airtel Payments Bank,99028476,Please do eKYC before initiating transaction,NEW DELHI,Delhi,110086
4,2819199185,500.0,Fail,Fingpay,2021-11-02 08:07:37,Airtel Payments Bank,20390048,Insufficient fund,Muzaffarpur,Bihar,843117
...,...,...,...,...,...,...,...,...,...,...,...
418381,2838914225,1010.0,Success,FINO,2021-11-28 15:23:13,Yes Bank,10051001,AEPS Transaction Success,,,
418382,2839392313,3000.0,Success,FINO,2021-11-29 10:25:10,Yes Bank,99010442,AEPS Transaction Success,Gurgaon,Haryana,122001
418383,2839554684,5000.0,Success,FINO,2021-11-29 13:07:08,Yes Bank,99056053,AEPS Transaction Success,Rajkot,Gujarat,360020
418384,2839899838,1000.0,Success,FINO,2021-11-29 19:36:30,Yes Bank,21810000,AEPS Transaction Success,Delhi,Delhi,110037


In [27]:
df2.to_csv('aeps_total_nov21.csv', index=False)

In [ ]:
df2[df2.duplicated(subset='ekotrxnid')]

In [ ]:
df2['month'] = pd.to_datetime(df2['date']).dt.month

In [24]:
df2.groupby(['month'])['amount'].sum().reset_index().to_csv('aeps_total_gtv.csv', index=False)

In [23]:
df2.to_csv('AePS_failed_trxns.csv', index=False)

In [18]:
df2.pivot(index='bank_name', columns=['month', 'destination'], values='amount')

ValueError: Index contains duplicate entries, cannot reshape

In [18]:
df2[(df2['bank_name'] == 'Bank of Baroda') | (df2['bank_name'] == 'FINO Payments Bank')]

,ekotrxnid,amount,status,destination,date,bank_name,message
22,2799122637,1000.0,Fail,FINO,2021-10-01,Bank of Baroda,"Suspected fraud, decline / Transactions declin..."
23,2799126293,0.0,Fail,FINO,2021-10-01,Bank of Baroda,Invalid Aadhaar Number
24,2799126526,0.0,Fail,FINO,2021-10-01,Bank of Baroda,"Suspected fraud, decline / Transactions declin..."
68,2799146169,0.0,Fail,FINO,2021-10-01,Bank of Baroda,Freeze or Frozen Account
75,2799147551,0.0,Fail,FINO,2021-10-01,Bank of Baroda,"Suspected fraud, decline / Transactions declin..."
...,...,...,...,...,...,...,...
862491,2867454129,1000.0,Fail,FingPay,2021-12-31,Bank of Baroda,Suspected Fraud
862493,2867454645,0.0,Fail,FingPay,2021-12-31,FINO Payments Bank,"Fingerprint did not matched with Aadhaar, plea..."
862498,2867463547,0.0,Fail,FingPay,2021-12-31,Bank of Baroda,Selected Banks not supported for this transaction
862499,2867463799,1400.0,Fail,FingPay,2021-12-31,Bank of Baroda,Transaction is restircted from 23:30 to 5:0. P...


In [8]:
df2.to_csv("AePS_total_trxns.csv", index = False)

In [3]:
# fund settlement failed tx
query_1 = f"select wt.holdtransactionid as ekotrxnid, wt.amount, wt.status, date(wt.txtime) as date, \
wt.narration as message from wallettransaction wt \
left join wallettransactionstatus wts on wt.status = wts.statusid \
where wt.txtime >= '2021-10-01 00:00:00' and wt.txtime < '2022-01-01 00:00:00' and wt.typeid = 392 and wt.status = 3 \
order by ekotrxnid"
df3 = pd.read_sql(query_1, conn_mysql)

In [6]:
# fund settlement total tx
query_1 = f"select distinct(wt.holdtransactionid) as ekotrxnid, wt.amount, wt.status, wt.txtime, \
csp.cspcode, wt.narration as message from wallettransaction wt \
left join wallettransactionstatus wts on wt.status = wts.statusid \
left join ekocsp csp on wt.originationid = csp.customerid \
where wt.txtime >= '2021-11-01 00:00:00' and wt.txtime < '2022-02-01 00:00:00' and wt.typeid = 392 \
order by ekotrxnid"
df3 = pd.read_sql(query_1, conn_mysql)
df3

,ekotrxnid,amount,status,txtime,cspcode,message
0,2818164819,19000.0,1,2021-11-01 00:14:33,99070041,Completed
1,2818190886,7699.0,1,2021-11-01 05:28:19,99106110,Completed
2,2818197460,14000.0,1,2021-11-01 06:25:44,99101674,Completed
3,2818199017,24999.0,1,2021-11-01 06:39:38,24401001,Completed
4,2818202257,200000.0,1,2021-11-01 06:57:25,101110001,Completed
...,...,...,...,...,...,...
46473,2887630419,500.0,1,2022-01-31 22:48:03,11541001,Completed
46474,2887631284,400.0,1,2022-01-31 22:55:26,11541001,Completed
46475,2887633502,1027.0,1,2022-01-31 23:16:08,99101194,Completed
46476,2887634138,10000.0,1,2022-01-31 23:22:26,99061797,Completed


In [10]:
df3[df3['status'] == 3]['message'].unique()

array(['Bad Request When any request elements fails schema validation',
       'Insufficient Balance in debit account',
       'Acquiring Bank CBS or node offline',
       'Rejected/Failed at beneficiary bank', 'Request Not Found',
       'Request not posted to Yesbank', 'Evalue given manually', 'Failed',
       'Rejected/Failed at upstream IMPS Service', 'Account closed',
       'Account blocked/frozen', 'Beneficiary not acceptable',
       'Invalid IFSC Code', 'Amount limit exceeded',
       'Invalid Beneficiary MMID/Mobile Number',
       'Transaction in progress',
       'IMPS is not enabled for the beneficiary IFSC',
       'Amount reversed due to system issue CS-23756',
       'Rejected/Failed at upstream CBS Service', 'Request time out'],
      dtype=object)

In [11]:
df3 = df3.replace({'status':{1:'Success', 3:'Failure'}})

In [14]:
df3 = df3.merge(df, on='cspcode', how='left').drop_duplicates(subset='ekotrxnid')
df3

,ekotrxnid,amount,status,txtime,cspcode,message,city,state,zip
0,2818164819,19000.0,Success,2021-11-01 00:14:33,99070041,Completed,Jaunpur,Uttar Pradesh,223101
1,2818190886,7699.0,Success,2021-11-01 05:28:19,99106110,Completed,Sitamarhi,Bihar,843325
2,2818197460,14000.0,Success,2021-11-01 06:25:44,99101674,Completed,East Champaran,Bihar,845420
3,2818199017,24999.0,Success,2021-11-01 06:39:38,24401001,Completed,Jhalawar,Rajasthan,326033
4,2818202257,200000.0,Success,2021-11-01 06:57:25,101110001,Completed,None,None,None
...,...,...,...,...,...,...,...,...,...
47002,2887630419,500.0,Success,2022-01-31 22:48:03,11541001,Completed,None,None,None
47003,2887631284,400.0,Success,2022-01-31 22:55:26,11541001,Completed,None,None,None
47004,2887633502,1027.0,Success,2022-01-31 23:16:08,99101194,Completed,Davangere,Karnataka,577001
47005,2887634138,10000.0,Success,2022-01-31 23:22:26,99061797,Completed,Lucknow,Uttar Pradesh,226025


In [15]:
df3.to_csv('fund_settlement_total.csv', index=False)